In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [2]:
idx2char = ['h', 'i', 'e', 'l', 'o']

In [3]:
# teach hihello : hihell --> ihello
x_data = [[0, 1, 0, 2, 3, 3]] # hihell
x_one_hot = [[
            [1, 0, 0, 0, 0], #h, 0
            [0, 1, 0, 0, 0], #i, 1
            [1, 0, 0, 0, 0], #h, 0
            [0, 0, 1, 0, 0], #e, 2
            [0, 0, 0, 1, 0], #l, 3
            [0, 0, 0, 1, 0]  #l, 3
            ]]
y_data = [[1, 0, 2, 3, 3, 4]] # ihello

In [4]:
np.shape(x_data)

(1, 6)

In [4]:
num_classes = 5
input_dim = 5 #one-hot size
hidden_size = 5 #output, 5 to directly predict one-hot
batch_size = 1 #one sentence
sequence_length =  6 #|ihello| = 6
learning_rate = 0.1

In [5]:
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, [None, sequence_length])

In [7]:
cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell,
    X, 
    initial_state=initial_state, 
    dtype= tf.float32)

In [8]:
# FC layer 
x_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(
        inputs=x_for_fc, num_outputs=num_classes, activation_fn=None)

In [11]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y:y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i, "loss:", l, "prediction:", result, "true Y:", y_data)
        
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print('\tPrediction str:', ''.join(result_str))

0 loss: 1.6234845 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: llllll
1 loss: 1.521307 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: llllll
2 loss: 1.4391664 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: llllll
3 loss: 1.3509946 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: llllll
4 loss: 1.2517651 prediction: [[3 0 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: lhllll
5 loss: 1.136398 prediction: [[3 0 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: lhllll
6 loss: 1.004584 prediction: [[3 0 3 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: lhlllo
7 loss: 0.86709803 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
8 loss: 0.7363381 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
9 loss: 0.621623 prediction: [[2 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ehello
10 loss: 0.52